In [325]:
import pandas as pd
import numpy as np
import time
import math
import datetime as dt
import statistics
import random
import heapq
from scipy import optimize
from scipy.stats import norm
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn import metrics
from scipy.ndimage import uniform_filter1d
import matplotlib.pyplot as plt
import os
import changefinder
from loguru import logger
from sklearn.preprocessing import StandardScaler
from itertools import combinations

logger.remove()
logger.add("C:\\Users\\52497\\Dropbox\\VIS\\Insight\\Code\\NBA.log", format="{time}{message}",rotation="500 MB")

30

In [326]:
dataset_list = ['carSales1', 'carSales2', 'Emission', 'Census', 'USBirth', 'AUSWeather', 'NBA', 'COVID-19']
data_index = 2
dataset = dataset_list[data_index]
folder_path = "C:\\Users\\52497\\Dropbox\\VIS\\Insight\\"
data_folder = folder_path + "Data\\"
result_folder = folder_path + "res\\"

raw_data_path = os.path.join(data_folder, '{}.csv'.format(dataset))
insight_data_path = os.path.join(result_folder, 'insights_{}.csv'.format(dataset))

data = pd.read_csv(raw_data_path)
data = data.loc[(data!=0).any(axis=1)].dropna()

In [327]:
def subspace_ordering(feature_measures, df):
    feature_unique_value_matrix = dict(zip(feature_measures,
                                       [df[feature].nunique() for feature in feature_measures]))
    sorted_feature_dict = {k: v for k, v in sorted(feature_unique_value_matrix.items(), key=lambda item: item[1], reverse=False)}
    return list(sorted_feature_dict.keys())

In [328]:
data

,Year,State,Producer Type,Energy Source,CO2 (kt),SO2 kt,NOx kt
0,1990,AK,Commercial Cogen,Coal,821.929,13.191,3.009
1,1991,AK,Commercial Cogen,Coal,848.745,8.359,3.146
2,1992,AK,Commercial Cogen,Coal,860.878,8.469,3.195
3,1993,AK,Commercial Cogen,Coal,858.244,8.393,3.208
4,1994,AK,Commercial Cogen,Coal,866.661,8.880,3.238
...,...,...,...,...,...,...,...
41151,2014,WY,Utility Sector Non-Cogen,Natural Gas,27.760,0.000,0.056
41152,2015,WY,Utility Sector Non-Cogen,Natural Gas,0.395,0.000,0.000
41153,2005,WY,Utility Sector Non-Cogen,Petroleum,0.000,0.002,0.023
41154,2006,WY,Utility Sector Non-Cogen,Petroleum,0.317,0.002,0.027


In [329]:
def point_power_law(phi):
    """
    :param: phi
    :return: [breakdown_value, observation_value, predict_value]
    """
    ordered_phi = {k: v for k, v in sorted(phi.items(), key=lambda item: item[1], reverse=True)}
    keys = list(ordered_phi.keys())
    values = list(ordered_phi.values())
    # print(values)
    ydata = values[1::]
    ydata = [i for i in ydata if i != 0]
    if len(values) < 4:
        return None
    xdata = range(2, len(ydata) + 2)
    logx = np.log10(xdata)
    logy = np.log10(ydata)
    pinit = [1.0]

    fitfunc = lambda p, x: p[0] - 0.7 * x
    errfunc = lambda p, x, y: (y - fitfunc(p, x))
    powerLawFunc = lambda amp, index, x: amp * (x ** index)
    try:
        out = optimize.leastsq(errfunc, pinit, args=(logx, logy), full_output=1)

        pfinal = out[0]
        covar = out[1]

        index = - 0.7
        amp = 10.0 ** pfinal[0]
        # indexErr = np.sqrt(covar[1][1])
        # ampErr = np.sqrt(covar[0][0]) * amp


        # predict_errs = ([1] + ydata) - powerLawFunc(amp, index, range(1, len(ydata) + 2))
        # print(predict_errs)
        # mu, std = norm.fit(predict_errs)
        # plt.hist(predict_errs, density=True, alpha=0.6, color='g')
        # plt_xmin, plt_xmax = plt.xlim()
        # plt_x = np.linspace(plt_xmin, plt_xmax, 100)
        # p = norm.pdf(plt_x, mu, std)
        # plt.plot(plt_x, p, 'k', linewidth=2)
        # title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
        # plt.title(title)
        # plt.show()
        residuals = []
        for idx, val in enumerate(xdata):
            residuals.append(ydata[idx] - powerLawFunc(amp, index, xdata[idx]))
        mean,std=norm.fit(residuals)
        sig = norm.cdf(values[0] - powerLawFunc(amp, index, 1), mean, std)
        # plt.bar(range(1,len(keys)+1), values)
        # plt.title("SIG: " + str(sig) + "val: " + str(values[0] - powerLawFunc(amp, index, 1)))
        # predict_res = [powerLawFunc(amp, index, i) for i in range(1,len(keys)+1)]
        # plt.plot(range(1,len(keys)+1), predict_res, color='red')
        # plt.show()
        # print(predict_err, sig)
        #  [breakdown_value, sig]
        return [keys[0], sig]

    except:
        return [keys[0], -1]

In [330]:
def calc_top1_insight(grouped_df, breakdown, measure):
    global sum_impact
    breakdown_measure_dict = dict(zip(list(grouped_df[breakdown]),list(grouped_df[measure])))
    res = point_power_law(breakdown_measure_dict)

    if not res: return None

    result_dict = dict(zip(
    ['breakdown','breakdown_value', 'sig'],
    [breakdown] + res ))
    return result_dict

def get_subspace_df(subspace_condition_dict, df):
    condition = pd.Series(True, index=df.index)
    for feature in subspace_condition_dict:
        if subspace_condition_dict[feature] == '*':
            continue
        condition = condition & (df[feature] == subspace_condition_dict[feature])
    return df[condition]

def calc_top1_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    calc_dict = calc_top1_insight(grouped_df, breakdown, measure)
    if not calc_dict: return None
    res_dict = dict(zip(['impact', 'insight', 'insight_type', 'score', 'measure'],
                        [impact, 'top1', 'point', calc_dict.get('sig')*impact, measure]))
    top1_insight = dict(subspace_condition_dict, **calc_dict, **res_dict)
    return top1_insight

def calc_trend_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global time_col
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    #  Calculate trend insight
    try:
        x = grouped_df[breakdown].map(dt.datetime.toordinal).values #
    except:
        x = grouped_df[breakdown].values #
    y = grouped_df[measure].values #
    x = x.reshape(-1, 1) #
    y = y.reshape(-1, 1) #
    if x.shape[0] < 2: return None
    reg = LinearRegression().fit(x, y) #
    slope = reg.coef_[0][0] #
    r2_score = reg.score(x, y)
    # sig = r2_score * norm.cdf(abs(slope), 0.2, 10000)
    sig = r2_score
    # grouped_df.plot(x=breakdown, y=measure)
    # plt.plot(grouped_df[breakdown], reg.predict(x))
    # plt.title("Sig Score is: {}, subspace condition: {}".format(sig, subspace_condition_dict))
    # plt.show()
    # print(subspace_condition_dict, "Sig: ", sig, "Impact: ", impact)
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [measure] + [sig, impact] + ['trend', 'shape', sig*impact]))
    trend_insight = dict(subspace_condition_dict, **result_dict)
    return trend_insight

def calc_change_point_abs(y_MA, MA_size):
    y_change = []
    for index in range(len(y_MA)):
        if index - MA_size < 0 or index + MA_size > len(y_MA) - 1:
            y_change.append(0)
        else:
            y_change.append(abs(y_MA[index - round(MA_size / 2.0)] - y_MA[index + round(MA_size / 2.0)]))
    return y_change

def calc_change_point_sigma(y_MA, y_MA_square, MA_size):
    y_sigma = []
    for i in range(len(y_MA)):
        if i - MA_size < 0 or i + MA_size > len(y_MA) - 1:
            y_sigma.append(1)
        else:
            left_part = (y_MA_square[i-round(MA_size/2.0)] + y_MA_square[i+round(MA_size/2.0)]) / (2.0 * MA_size)
            right_part = ((y_MA[i-round(MA_size/2.0)] + y_MA[i+round(MA_size/2.0)]) / (2.0 * MA_size)) ** 2
            y_sigma.append(math.sqrt((left_part - right_part) / MA_size))
        # if index == 0 or index == len(y) - 1:
        #     y_sigma.append(0)
        # elif 1 <= index < len(y) - 2:
        #     res = np.sqrt((y_MA_square[index-1] + y_MA_square[index+2])/2 - np.square(y_MA[index-1] + y_MA[index+2]) / 4)
        #     y_sigma.append(res)
        # elif index == len(y) - 2:
        #     res = np.sqrt((y_MA_square[index-1] + y[index+1]**2)/2 - np.square(y_MA[index-1] + y[index+1]) / 4)
        #     y_sigma.append(res)
    return y_sigma

def calc_change_point_insights_mean_value(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    MA_size = 5
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 20:
        return None
    y_MA = uniform_filter1d(y, size=MA_size, mode='nearest')
    y_MA_square = uniform_filter1d(np.square(y), size=MA_size, mode='nearest')
    y_change = calc_change_point_abs(y_MA, MA_size)
    y_sigma = calc_change_point_sigma(y_MA=y_MA, y_MA_square=y_MA_square, MA_size=MA_size)
    k_mean = [y_change[i] / y_sigma[i] for i in range(len(y))]
    k_mean_max = max(k_mean)
    k_mean_max_index = k_mean.index(k_mean_max)
    sig = norm.cdf(k_mean_max * np.square(MA_size))
    breakdown_value = x[k_mean_max_index]
    # print("The significance is {}, with value = {}".format(sig, breakdown_value))
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [measure] +[sig, impact]
                           + ['change point', 'shape', sig*impact]))
    change_point_insight = dict(subspace_condition_dict, **result_dict)
    # print(k_mean)
    # print(k_mean.index(k_mean_max))
    # if sig > 0.8:
    #     print("-----------------------------")
    #     plt.plot(x,y)
    #     plt.plot([breakdown_value], [y[k_mean.index(k_mean_max)]],color='red', marker='o', markersize=3)
    #     plt.show()
    if not sig:
        return None
    return change_point_insight

def calc_change_point_insights_slope(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    MA_size = 2
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 4:
        return None
    # Stores the slope between x_i-1 and x_i on slope_list[i]
    modified_y = [(y[i]-min(y))/(max(y)-min(y)) for i in range(0, len(y))]
    slope_list = [(modified_y[i]-modified_y[i-1]) if i != 0 else 0 for i in range(0, len(y))]
    slope_diff_abs = [abs(slope_list[i]-slope_list[i+1]) if i != 0 else 0 for i in range(0, len(y)-1)] + [0]
    # sigma = [0.0] + [math.sqrt((y[i-1]**2 + y[i]**2 + y[i+1]**2)/6 - (y[i-1]+y[i]+y[i+1])**2 / (4*9))/math.sqrt(3)
    #          for i in range(1, len(y)-1)] + [0.0]
    # k_slope = [slope_diff_abs[i]/sigma[i] if sigma[i]!=0 else 0 for i in range(0, len(y))]
    k_slope = [slope_diff_abs[i] for i in range(0, len(y))]
    k_slope_max = max(k_slope)
    sig = norm.cdf(k_slope_max)
    breakdown_value = x[k_slope.index(k_slope_max)]
    # print("The significance is {}, with value = {}".format(sig, breakdown_value))
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [measure] + [sig, impact] + ['change point', 'shape', sig*impact]))
    change_point_insight = dict(subspace_condition_dict, **result_dict)
    # plt.title('Sig: '+ str(sig)+" slope left: "+ str(slope_list[k_slope.index(k_slope_max)]/max(y))+" slope right: "+ str(slope_list[k_slope.index(k_slope_max)+1]/max(y)))
    # plt.plot(x,y)
    # plt.plot([breakdown_value], [y[k_slope.index(k_slope_max)]],color='red', marker='o', markersize=3)
    # plt.show()
    if not sig:
        return None
    return change_point_insight

def calc_change_point_insights_changefinder(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 4:
        return None
    #CHANGEFINDER PACKAGE
    # f, (ax1, ax2) = plt.subplots(2, 1)
    # f.subplots_adjust(hspace=0.4)
    # ax1.plot(y)
    # ax1.set_title("data point")
    #Initiate changefinder function
    cf = changefinder.ChangeFinder()
    scores = [cf.update(int(p)) for p in y]
    # ax2.plot(scores)
    # ax2.set_title("anomaly score")
    # plt.show()
    score_max = max(scores)
    sig = norm.cdf(score_max/15.0)
    breakdown_value = x[scores.index(score_max)]
    # print("The significance is {}, with value = {}".format(sig, breakdown_value))
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [measure] + [sig, impact] + ['change point', 'shape', sig*impact]))
    change_point_insight = dict(subspace_condition_dict, **result_dict)
    if not sig:
        return None
    return change_point_insight

def calc_outlier_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 4:
        return None
    mean = np.mean(y)
    std = np.std(y)
    z_score = (y - mean) / std
    z_score_max_index = np.argmax(z_score)
    z_score_max = z_score[z_score_max_index]
    sig = norm.cdf(z_score_max)
    breakdown_value = x[z_score_max_index]
    if math.isnan(sig) :
        return None
    # print("The significance is {}, with value = {}, kmeans is {}".format(sig, breakdown_value, z_score_max))
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [measure] + [sig, impact] + ['outlier', 'shape', sig*impact]))
    outlier_insight = dict(subspace_condition_dict, **result_dict)
    # print("-----------------------------")
    # plt.plot(x,y)
    # plt.plot([breakdown_value], [y[z_score_max_index]],color='red', marker='o', markersize=3)
    # plt.show()
    return outlier_insight

def calc_attribution_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if np.sum(y) == 0:
        return None
    portion = np.max(y)/np.sum(y)


    sig = norm.cdf(portion, 0.5, 0.3)
    breakdown_index = np.argmax(y)
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, x[breakdown_index]] + [measure] + [sig, impact] + ['attribution', 'point', sig*impact]))
    attribution_insight = dict(subspace_condition_dict, **result_dict)
    return attribution_insight

def calc_evenness_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) == 1: return None
    std = np.std(y)
    sig = 2 * (1 - norm.cdf(std, 0, 3))
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [sig, impact] + [measure] + ['evenness', 'point', sig*impact]))
    evenness_insight = dict(subspace_condition_dict, **result_dict)
    return evenness_insight

def calc_correlation_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact,
                              corr_subspace_condition_dict, corr_subspace_df):
    global time_col
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    if corr_subspace_df.shape[0] == 0 or corr_subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    corr_grouped_df = corr_subspace_df.groupby(breakdown, as_index=False).sum()
    #  Calculate trend insight
    x = grouped_df[breakdown].values
    if len(x) != len(corr_grouped_df[breakdown].values) \
            or not (x == corr_grouped_df[breakdown].values).all()\
            or len(x) < 2:
        logger.info("data not matched")
        return None
    y1 = grouped_df[measure].values
    y2 = corr_grouped_df[measure].values
    sig, _ = pearsonr(y1, y2)
    return sig

def calc_cross_measure_correlation_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    measure_val_x = grouped_df[measures[0]].values.reshape(-1,1)
    measure_val_y = grouped_df[measures[1]].values.reshape(-1,1)
    if measure_val_x.shape[0] < 10: return None
    reg = LinearRegression().fit(measure_val_x, measure_val_y) #
    slope = reg.coef_[0][0] #
    r2_score = reg.score(measure_val_x, measure_val_y)
    sig = r2_score
    # if sig > 0.5:
    #     plt.title('Breakdown: ' + breakdown + "Sig: " + str(sig))
    #     plt.scatter(measure_val_x, measure_val_y)
    #     measure_y_pred = reg.predict(measure_val_x)
    #     plt.plot(measure_val_x, measure_y_pred)
    #     plt.show()
    # else:
    #     print("Sig is: ", sig)
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [';'.join(measures)] + [sig, impact] +  ['cross measure correlation', 'compound', sig*impact]))
    cross_insight = dict(subspace_condition_dict, **result_dict)
    return cross_insight


from itertools import cycle
from sklearn.cluster import MeanShift, estimate_bandwidth
import collections
from sklearn.cluster import DBSCAN

def calc_clustering_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    measure_val_x = np.array(grouped_df[measures[0]].values)
    measure_val_y = np.array(grouped_df[measures[1]].values)
    if measure_val_x.shape[0] < 30: return None
    X = np.vstack((measure_val_x, measure_val_y)).T
    X_scale = StandardScaler().fit_transform(X)

    # Compute DBSCAN
    db = DBSCAN(eps=0.3, min_samples=5).fit(X_scale)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    counters = collections.Counter(labels)
    max_group = max(counters, key=counters.get)
    max_total_ratio = counters.get(max_group) * 1.0 / len(labels)

    # #############################################################################
    # Plot result
    if n_clusters_ == 1:
        score = 1
    elif n_clusters_ == 0:
        return None
    else:
        score = 0.5 * (metrics.silhouette_score(X, labels) + 1)
    # if score > 0.5 and max_total_ratio > 0.8:
    #     plt.figure(1)
    #     plt.clf()
    #
    #     colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
    #     for k, col in zip(range(n_clusters_), colors):
    #         my_members = labels == k
    #         plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    #             # plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
    #             #          markeredgecolor='k', markersize=14)
    #     plt.title('Estimated number of clusters: %d, Score: %0.2f, Size: %d' % (n_clusters_, score, len(measure_val_x)))
    #     plt.show()
    result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [';'.join(measures)] + [score, impact]  + ['clustering', 'compound shape',
                                                                (score + max_total_ratio)*impact*0.5]))
    clustering_insight = dict(subspace_condition_dict, **result_dict)
    return clustering_insight

In [331]:
def traverse_compound_insights(root, df, measure, feature_names, breakdown, top_num, output_array,
                               subspace_condition_dict, calc_insights,
                               subspace_df, impact):
    if len(output_array) == top_num and impact < output_array[0][0] :
        # print('Subspace pruned')
        return output_array

    subspaces = []
    for col_index, val_index in enumerate(root):
        if feature_names[col_index] == breakdown and val_index == '*':
            available_bk_val = subspace_df[feature_names[col_index]].unique().tolist() + ['*']
            subspaces += (generate_array(root, col_index, val) for val in available_bk_val)
    # print('subspace: ', root)
    # print('breakdown: ', breakdown)
    # print('subspaces: ', subspaces)
    subspace_comb = combinations(subspaces, 2)
    for comb in list(subspace_comb):
        sub0 = comb[0]
        sub1 = comb[1]
        sub_diff = []
        for idx, _ in enumerate(sub0):
            if sub0[idx] != sub1[idx]:
                sub_diff = [sub0[idx], sub1[idx]]

        corr_subspace_df1 = get_subspace_df(dict(zip(feature_names, sub1)), subspace_df)
        corr_subspace_condition_dict1 = dict(zip(feature_names, sub1))
        corr_impact1 = corr_subspace_df1[measure].count() / sum_impact

        corr_subspace_df0 = get_subspace_df(dict(zip(feature_names, sub0)), subspace_df)
        corr_subspace_condition_dict0 = dict(zip(feature_names, sub0))
        corr_impact0 = corr_subspace_df0[measure].count() / sum_impact

        sig = calc_insights(corr_subspace_condition_dict0, time_col, measure, corr_subspace_df0,
                        impact,corr_subspace_condition_dict1,
                        corr_subspace_df1)
        if '*' in sub_diff:
            impact = max(corr_impact0, corr_impact1)
        else:
            impact = corr_impact0 + corr_impact1
        if sig:
            result_dict = dict(zip(['breakdown','breakdown_value', 'measure', 'sig', 'impact', 'insight', 'insight_type',
                'score'],
               [breakdown, str(sub_diff[0]) + ";" + str(sub_diff[1]), measure] + [sig, impact]
                                   + ['correlation', 'compound shape', sig*impact]))
            insight = dict(subspace_condition_dict, **result_dict)
            logger.info("Has insight")
            if len(output_array) < top_num:
                heapq.heappush(output_array, (insight.get('score'), random.random(), insight))
                logger.info("Current Score LB: {}".format(output_array[0][0]))
            elif output_array[0][0] < insight.get('score'):
                heapq.heapreplace(output_array, (insight.get('score'), random.random(), insight))
                logger.info("Current Score LB: {}".format(output_array[0][0]))

    return output_array

In [332]:
def generate_array(arr, index, val):
    new_arr = arr[:]
    new_arr[index] = val
    return new_arr

def generate_process_node(feature_names, measure, output_array, df,
                          calc_insights = lambda: "Need to define", breakdowns = [], top_num = 50):
    def process_node(node, result_stack, breakdown):
        nonlocal output_array
        # subspace_condition_dict. e.g., ['Year', 'Brand', 'Category'] ['2007', '*', '*']
        subspace_df = get_subspace_df(dict(zip(feature_names, node)), df)
        if type(measure) == str:
            impact = subspace_df[measure].count() / sum_impact
        else:
            impact = 0.5 * (subspace_df[measure[0]].count() / sum_impact0 + subspace_df[measure[1]].count() / sum_impact1)
        subspace_condition_dict = dict(zip(feature_names, node))

        if len(output_array) == top_num and output_array[0][0] > impact:
            logger.info("Pruned %f, subspace: %s"%(impact, subspace_condition_dict))
            return False
        logger.info("Impact not pruned %f, subspace: %s"%(impact, subspace_condition_dict))
        if calc_insights == calc_correlation_insights:
            if breakdown != time_col:
                output_array = traverse_compound_insights(root=node,
                                                          df=df,
                                                          feature_names=feature_names,
                                                          breakdown=breakdown,
                                                          measure=measure,
                                                          top_num=top_num, output_array=output_array,
                                                          subspace_condition_dict=subspace_condition_dict,
                                                          calc_insights=calc_insights,
                                                          subspace_df=subspace_df, impact=impact)
        else:
            insight = calc_insights(subspace_condition_dict=subspace_condition_dict,
                         breakdown=breakdown, measure=measure, impact = impact, subspace_df=subspace_df)
            if insight:
                # output.append(insight)
                if len(output_array) < top_num:
                    modified_insight = dict(insight)
                    heapq.heappush(output_array, (insight.get('score'), random.random(), modified_insight))
                    logger.info("Current Score LB: {}".format(output_array[0][0]))
                elif output_array[0][0] < insight.get('score'):
                    modified_insight = dict(insight)
                    heapq.heapreplace(output_array, (insight.get('score'), random.random(), modified_insight))
                    logger.info("Current Score LB: {}".format(output_array[0][0]))
            else:
                # print("None Insight, subspace: %s, breakdown: %s"%(subspace_condition_dict, breakdown))
                return False
        if node.count('*') == 1:
            return False
        return True
    return process_node

def BFS_tranverse_and_process(df, feature_names, process_node = lambda x: True, type=None):
    feature_unique_value_matrix = [df[feature].unique().tolist() for feature in feature_names]
    breakdown_list = feature_names.copy()

    for breakdown in breakdown_list:
        traverse_root = ['*' for i in feature_names] + [[i for (i, val) in enumerate(feature_names) if val == breakdown]]
        result_stack = [traverse_root]

        while len(result_stack) > 0:
            *root, ban = result_stack.pop(0)
            result = process_node(node=root, result_stack=[[*root, ban]]+ result_stack, breakdown=breakdown)
            if not result:
                continue
            banned_index = ban[:]
            subspace_df = get_subspace_df(dict(zip(feature_names, root)), df)
            for col_index, val_index in enumerate(root):
                if col_index in banned_index:
                    continue
                banned_index.append(col_index)
                banned_index = list(set(banned_index))
                available_features = subspace_df[feature_names[col_index]].unique().tolist()
                if val_index == '*':
                    result_stack += ([generate_array(root, col_index, val)+[banned_index[:]]  for val in available_features])

In [333]:
start_time = time.time()


# CarSales1.csv
# feature_names = ['Year', 'Brand', 'Category']
# measures = ['Sales']
# time_col = 'Year'
# data[time_col] = data[time_col].apply(lambda year_string: year_string.split('/')[0])


# carSales2.csv
# feature_names = ['Year', 'Brand', 'Body', 'Engine Type', 'Model', 'Registration']
# # measure = 'Price'
# measure = 'Mileage'
# time_col = 'Year'


# Census.csv
# feature_names = ['Birthday', 'Age Segment', 'Marital Status', 'Sex', 'Age Group']
# measure = 'Count of persons'
# time_col = 'Birthday'
#
# data[time_col] = data[time_col].apply(lambda year_string: year_string.split('/')[0])


# Emission.csv
feature_names = ['Year', 'State', 'Producer Type', 'Energy Source']
measures = ['CO2 (kt)', 'SO2 kt']
time_col = 'Year'

# USBirth.csv
# feature_names = ['Year', 'Age', 'Birth order']
# measure = 'Value'
# time_col = 'Year'

# # AUSWeather.csv
# feature_names = ['Date','Location','RainToday','RainTomorrow']
# measures = ['MinTemp','MaxTemp']
# measure = 'MinTemp'
# time_col = 'Date'
#
# NBA.csv
# feature_names = ['name','year','team_name','age','lg_name','pos_name']
# measures = ['PTS','AST']
# measure = 'PTS'
# time_col = 'year'

# COVID-19.csv
# feature_names = ['date','county','state']
# measures = ['cases', 'deaths']
# measure = 'cases'
# time_col = 'date'
# data[time_col] = pd.to_datetime(data[time_col]).dt.date

res = []
for measure in measures:
    sum_impact = data[measure].count()
    feature_names = subspace_ordering(feature_measures=feature_names, df=data)
    time_col_index = feature_names.index(time_col)

    output = []
    process_node_trend = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_trend_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_trend)
    print("Trend finished")
    res += [out[-1] for out in output.copy()]
    print(res[0])

    output = []
    process_node_top1 = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_top1_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_top1)
    print("Top1 finished")
    res += [out[-1] for out in output.copy()]

    output = []
    #
    process_node_change_point = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_change_point_insights_mean_value)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_change_point)
    print("Change point finished")
    res += [out[-1] for out in output.copy()]

    output = []
    process_node_outlier = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_outlier_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_outlier)
    print("Outlier finished")
    res += [out[-1] for out in output.copy()]

    output = []
    process_node_attribution = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_attribution_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_attribution)
    print("Attribution finished")
    res += [out[-1] for out in output.copy()]

    output = []
    process_node_correlation = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measure,
                                         calc_insights=calc_correlation_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_correlation, type='time breakdown')
    print("Correlation finished")
    res += [out[-1] for out in output.copy()]

    output = []

if len(measures) > 1:
    sum_impact0 = data[measures[0]].count()
    sum_impact1 = data[measures[1]].count()
    feature_names = subspace_ordering(feature_measures=feature_names, df=data)
    time_col_index = feature_names.index(time_col)
    process_node_clustering = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measures,
                                         calc_insights=calc_clustering_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_clustering)
    print("Clustering finished")
    res += [out[-1] for out in output.copy()]

    output = []
    process_node_cross_measure = generate_process_node(feature_names = feature_names,
                                         output_array=output,
                                         df = data,
                                         measure = measures,
                                         calc_insights=calc_cross_measure_correlation_insights)

    BFS_tranverse_and_process(data,feature_names, process_node=process_node_cross_measure)
    print("Cross measure finished")
    res += [out[-1] for out in output.copy()]

    output = []


df = pd.DataFrame(res)
# df['measure'] = measure
# df['measure'][df.insight == 'cross measure correlation'] = ';'.join('measures')
# df['measure'][df.insight == 'clustering'] = ';'.join(measures)
end_time = time.time()
print("Time Elapsed: {}".format(end_time-start_time ))

Trend finished
{'Producer Type': '*', 'Energy Source': '*', 'Year': '*', 'State': 'MS', 'breakdown': 'Year', 'breakdown_value': -1, 'measure': 'CO2 (kt)', 'sig': 0.561614612567644, 'impact': 0.014724463018757896, 'insight': 'trend', 'insight_type': 'shape', 'score': 0.008269473593546317}
Top1 finished
Change point finished
Outlier finished
Attribution finished
Correlation finished
Trend finished
{'Producer Type': '*', 'Energy Source': '*', 'Year': '*', 'State': 'MS', 'breakdown': 'Year', 'breakdown_value': -1, 'measure': 'CO2 (kt)', 'sig': 0.561614612567644, 'impact': 0.014724463018757896, 'insight': 'trend', 'insight_type': 'shape', 'score': 0.008269473593546317}
Top1 finished
Change point finished
Outlier finished
Attribution finished
Correlation finished
Clustering finished
Cross measure finished
Time Elapsed: 189.3702461719513


<ipython-input-330-28dac05bc291>:106: RuntimeWarning: invalid value encountered in double_scalars
  k_mean = [y_change[i] / y_sigma[i] for i in range(len(y))]
<ipython-input-330-28dac05bc291>:205: RuntimeWarning: invalid value encountered in true_divide
  z_score = (y - mean) / std
C:\Users\52497\.conda\envs\py38\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
C:\Users\52497\.conda\envs\py38\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
C:\Users\52497\.conda\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1844: RuntimeWarning: divide by zero encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
<ipython-input-330-28dac05bc291>:106: RuntimeWarning: invalid val

In [334]:
df

,Producer Type,Energy Source,Year,State,breakdown,breakdown_value,measure,sig,impact,insight,insight_type,score
0,*,*,*,MS,Year,-1,CO2 (kt),0.561615,0.014724,trend,shape,0.008269
1,Industrial Cogen,Other,*,*,Year,-1,CO2 (kt),0.300003,0.029060,trend,shape,0.008718
2,*,*,*,IA,Year,-1,CO2 (kt),0.494322,0.020945,trend,shape,0.010353
3,*,*,*,UT,Year,-1,CO2 (kt),0.565674,0.015551,trend,shape,0.008797
4,*,*,*,CO,Year,-1,CO2 (kt),0.521803,0.017251,trend,shape,0.009002
...,...,...,...,...,...,...,...,...,...,...,...,...
695,Electric Utility,Coal,*,*,Year,-1,CO2 (kt);SO2 kt,0.650993,0.051414,cross measure correlation,compound,0.033470
696,Electric Utility,*,*,*,State,-1,CO2 (kt);SO2 kt,0.680802,0.211148,cross measure correlation,compound,0.143750
697,Industrial Non-Cogen,*,*,*,Year,-1,CO2 (kt);SO2 kt,0.560324,0.057732,cross measure correlation,compound,0.032348
698,*,*,2014,*,State,-1,CO2 (kt);SO2 kt,0.724247,0.043590,cross measure correlation,compound,0.031570


In [335]:
print("Time Elapsed: {}".format(end_time-start_time ))

Time Elapsed: 189.3702461719513


In [336]:
"""
print point insight result
"""
# for insight in res:
#     subspace_cond = {k:insight[k] for k in insight.keys() if k in feature_names}
#     condition = dict(subspace_cond)
#     selected_df = get_subspace_df(condition, data)
#     grouped_df = selected_df.groupby(insight['breakdown'], as_index=False).sum()
#     if insight['insight'] == 'top1':
#         grouped_df = grouped_df.sort_values(by=[measure])
#         grouped_df.plot(x=insight['breakdown'], y=measure, kind='bar')
#     elif insight['insight'] == 'trend' or insight['insight'] == 'evenness':
#         grouped_df = grouped_df.sort_values(by=time_col)
#         grouped_df.plot(x=insight['breakdown'], y=measure, kind='line')
#     elif insight['insight'] == 'change point' or insight['insight'] == 'outlier':
#         grouped_df = grouped_df.sort_values(by=time_col)
#         y_val = grouped_df.loc[grouped_df[insight['breakdown']] == insight['breakdown_value']][measure]
#         plt.plot(grouped_df[insight['breakdown']], grouped_df[measure])
#         plt.plot([insight['breakdown_value']], [y_val.values[0]],color='red', marker='o', markersize=3)
#     elif insight['insight'] == 'attribution':
#         grouped_df = grouped_df.sort_values(by=[measure])
#         plt.pie(grouped_df[measure], labels = grouped_df[insight['breakdown']], startangle = 90,
#         counterclock = False)
#         plt.axis('square')
#     elif insight['insight'] == 'correlation':
#         grouped_df = selected_df.loc[selected_df[insight['breakdown']] == insight['breakdown_value'].split(';')[0]]
#         corr_grouped_df = selected_df.loc[selected_df[insight['breakdown']] == insight['breakdown_value'].split(';')[1]]
#         grouped_df = grouped_df.groupby(time_col, as_index=False).agg(
#                 {time_col: 'first', measure: 'sum'})
#         corr_grouped_df = corr_grouped_df.groupby(time_col, as_index=False).agg(
#                 {time_col: 'first', measure: 'sum'})
#         y1 = grouped_df[measure].values
#         y2 = corr_grouped_df[measure].values
#         print(grouped_df)
#         print(corr_grouped_df)
#         print(y1)
#         print(y2)
#         sig, _ = pearsonr(y1, y2)
#         print("Sig of correlation: ", sig)
#         plt.plot(grouped_df[time_col], grouped_df[measure], color='red')
#         plt.plot(corr_grouped_df[time_col], corr_grouped_df[measure], color='blue')
#     elif insight['insight'] == 'clustering':
#         measure_val_x = np.array(grouped_df[measures[0]].values)
#         measure_val_y = np.array(grouped_df[measures[1]].values)
#         X = np.vstack((measure_val_x, measure_val_y)).T
#         X_scale = StandardScaler().fit_transform(X)
#
#         # Compute DBSCAN
#         db = DBSCAN(eps=0.5, min_samples=5).fit(X_scale)
#         core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#         core_samples_mask[db.core_sample_indices_] = True
#         labels = db.labels_
#         # Number of clusters in labels, ignoring noise if present.
#         n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
#         n_noise_ = list(labels).count(-1)
#         counters = collections.Counter(labels)
#         max_group = max(counters, key=counters.get)
#         max_total_ratio = counters.get(max_group) * 1.0 / len(labels)
#         plt.figure(1)
#         plt.clf()
#
#         colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
#         for k, col in zip(range(n_clusters_), colors):
#             my_members = labels == k
#             plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
#             # plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
#             #          markeredgecolor='k', markersize=14)
#     elif insight['insight'] == 'cross measure correlation':
#         measure_val_x = np.array(grouped_df[measures[0]].values).reshape(-1,1)
#         measure_val_y = np.array(grouped_df[measures[1]].values).reshape(-1,1)
#         reg = LinearRegression().fit(measure_val_x, measure_val_y)
#         plt.scatter(measure_val_x, measure_val_y)
#         measure_y_pred = reg.predict(measure_val_x)
#         plt.plot(measure_val_x, measure_y_pred)
#     plt.title(condition)
#     plt.show()

'\nprint point insight result\n'

In [338]:
# df = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\Insight\\res\\insights_COVID-19.csv')
# for index, row in df.iterrows():
#     if row['breakdown'] == time_col and row['breakdown_value'] != '-1':
#         df.at[index, 'breakdown_value'] = str(pd.to_datetime(row['breakdown_value']).strftime("%Y-%m-%d"))
#     if row['date'] != '*':
#         df.at[index, 'date'] = str(pd.to_datetime(row['date']).strftime("%Y-%m-%d"))
#
with open(insight_data_path, 'w+') as file:
    df.to_csv(file, index=False, line_terminator='\n')
